The questions I am trying to answer:
1) What was the frequency of trade or how many trades per day
2) Average profit and loss from each trade
3) Whats the risk reward ratio
4) Average holding period per security

In [21]:
import pandas as pd
import numpy as np

In [22]:
with open('All for tony, thinkorswim trade challenge winner2.txt') as f:
    table = pd.read_table(f, sep='\s+', engine='python', header=None,   lineterminator='\n')
# sep=' |:', engine='python'

In [23]:
table.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,11/04/2016,02:21:43,PM,BOT,"+5,500",NEM,100,4,NOV,16,37.5,CALL,@.15,"$82,500.00"
1,11/04/2016,01:57:41,PM,SOLD,-300,UVXY,100,11,NOV,16,18.5,PUT,@2.25,"-$67,500.00"
2,11/04/2016,01:35:00,PM,BOT,+666,VXX,100,11,NOV,16,36.0,CALL,@3.40,"$226,440.00"
3,11/04/2016,01:27:59,PM,BOT,+200,TSLA,100,11,NOV,16,190.0,PUT,@4.60,"$92,000.00"
4,11/04/2016,01:07:58,PM,BOT,"+1,200",GPRO,100,11,NOV,16,11.5,PUT,@.70,"$84,000.00"


In [24]:
# table["1"]= table["2"] + table["3"]
table.iloc[:,0]=table.iloc[:,0]+" "+table.iloc[:,1]+""+table.iloc[:,2]
# table=table.drop(table.columns[[1,2]],axis=1)
table.iloc[:,5]=table.iloc[:,5]+" "+table.iloc[:,6].map(str)+" "+table.iloc[:,7].map(str)+" "+table.iloc[:,8].map(str)+" "+table.iloc[:,9].map(str)+" "+ \
table.iloc[:,10].map(str)+" "+table.iloc[:,11].map(str)
table=table.drop(table.columns[[1,2,6,7,8,9,10,11]],axis=1)
table.head()

,0,3,4,5,12,13
0,11/04/2016 02:21:43PM,BOT,"+5,500",NEM 100 4 NOV 16 37.5 CALL,@.15,"$82,500.00"
1,11/04/2016 01:57:41PM,SOLD,-300,UVXY 100 11 NOV 16 18.5 PUT,@2.25,"-$67,500.00"
2,11/04/2016 01:35:00PM,BOT,+666,VXX 100 11 NOV 16 36.0 CALL,@3.40,"$226,440.00"
3,11/04/2016 01:27:59PM,BOT,+200,TSLA 100 11 NOV 16 190.0 PUT,@4.60,"$92,000.00"
4,11/04/2016 01:07:58PM,BOT,"+1,200",GPRO 100 11 NOV 16 11.5 PUT,@.70,"$84,000.00"


In [25]:
import re
# table.iloc[:,13]=table.iloc[:,13].replace("@", "")
table.columns=['Time','Position','Quantity','Security','Price','Cash_flow']
# table["Price"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# table["Quantity"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# table["cash_flow"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# table.cash_flow = table.cash_flow.str.translate(None, "$") #use if u downgrading to python 2
# table.cash_flow = str(re.sub('[$]', '', str(table.cash_flow)))
table.Price =table.Price.str.replace('@','')
table.Price =table.Price.astype(float)
table.Cash_flow =table.Cash_flow.str.replace('$','')
table.Cash_flow =table.Cash_flow.str.replace(',','')
table.Cash_flow =table.Cash_flow.astype(float)
table.Quantity =table.Quantity.str.replace(',','')
table.Quantity = table.Quantity.astype(float)
table.head()

,Time,Position,Quantity,Security,Price,Cash_flow
0,11/04/2016 02:21:43PM,BOT,5500.0,NEM 100 4 NOV 16 37.5 CALL,0.15,82500.0
1,11/04/2016 01:57:41PM,SOLD,-300.0,UVXY 100 11 NOV 16 18.5 PUT,2.25,-67500.0
2,11/04/2016 01:35:00PM,BOT,666.0,VXX 100 11 NOV 16 36.0 CALL,3.40,226440.0
3,11/04/2016 01:27:59PM,BOT,200.0,TSLA 100 11 NOV 16 190.0 PUT,4.60,92000.0
4,11/04/2016 01:07:58PM,BOT,1200.0,GPRO 100 11 NOV 16 11.5 PUT,0.70,84000.0


In [26]:
# table.groupby('security').count()
# table.security.value_counts()

In [27]:
import datetime
import locale
locale.setlocale(locale.LC_ALL, "") # for datetime.strptime

# datetime_object = datetime.strptime('11/04/2016 02:21:43PM', '%x %I:%M:%S%p')
table.Time = pd.to_datetime(table.Time,format="%x %I:%M:%S%p")

table.head()

,Time,Position,Quantity,Security,Price,Cash_flow
0,2016-11-04 14:21:43,BOT,5500.0,NEM 100 4 NOV 16 37.5 CALL,0.15,82500.0
1,2016-11-04 13:57:41,SOLD,-300.0,UVXY 100 11 NOV 16 18.5 PUT,2.25,-67500.0
2,2016-11-04 13:35:00,BOT,666.0,VXX 100 11 NOV 16 36.0 CALL,3.40,226440.0
3,2016-11-04 13:27:59,BOT,200.0,TSLA 100 11 NOV 16 190.0 PUT,4.60,92000.0
4,2016-11-04 13:07:58,BOT,1200.0,GPRO 100 11 NOV 16 11.5 PUT,0.70,84000.0


In [47]:
# table.security.unique()
security = table.Security.unique()
average_holding_period =[0]* len(security)
total_profit =[0]* len(security)
average_profit =[0]* len(security)
number_of_trades=[0]* len(security)

trade_analyzis = {'security': security,'average_holding_period':[0]* len(security),\
                  'total_profit':[0]* len(security),'average_profit':[0]* len(security),'number_of_trades':[0]* len(security),\
                  'outstanding_quantity':[0]* len(security),'start_time':[0]* len(security),\
                  'direction':[0]* len(security)}


trade_analyzis = pd.DataFrame(trade_analyzis)
trade_analyzis.head()

,average_holding_period,average_profit,direction,number_of_trades,outstanding_quantity,security,start_time,total_profit
0,0,0,0,0,0,VXX 100 28 OCT 16 34.5 CALL,0,0
1,0,0,0,0,0,VXX 100 21 OCT 16 34.5 PUT,0,0
2,0,0,0,0,0,SPY 100 14 OCT 16 213.0 CALL,0,0
3,0,0,0,0,0,VXX 100 14 OCT 16 34.5 PUT,0,0
4,0,0,0,0,0,VXX 100 14 OCT 16 33.5 PUT,0,0


Security | Average Holding Period | Total Profit | Average Profit | Number of trades | 


In [53]:
table.sort_values(by='Time',inplace=True)
table

# z.index[0] + dt.timedelta(minutes=5)


,Time,Position,Quantity,Security,Price,Cash_flow
117,2016-10-11 10:10:19,BOT,200.0,VXX 100 28 OCT 16 34.5 CALL,1.50,30000.0
116,2016-10-11 13:50:28,SOLD,-200.0,VXX 100 28 OCT 16 34.5 CALL,1.86,-37200.0
115,2016-10-11 14:11:19,BOT,300.0,VXX 100 21 OCT 16 34.5 PUT,1.48,44400.0
114,2016-10-11 14:22:10,BOT,100.0,SPY 100 14 OCT 16 213.0 CALL,1.40,14000.0
113,2016-10-11 14:27:21,SOLD,-300.0,VXX 100 21 OCT 16 34.5 PUT,1.60,-48000.0
112,2016-10-11 14:30:09,BOT,10.0,VXX 100 14 OCT 16 34.5 PUT,1.00,1000.0
111,2016-10-11 14:31:19,BOT,390.0,VXX 100 14 OCT 16 34.5 PUT,1.00,39000.0
110,2016-10-11 14:43:11,SOLD,-100.0,SPY 100 14 OCT 16 213.0 CALL,1.42,-14200.0
109,2016-10-11 14:54:37,SOLD,-400.0,VXX 100 14 OCT 16 34.5 PUT,1.02,-40800.0
108,2016-10-12 12:46:13,BOT,1000.0,VXX 100 14 OCT 16 33.5 PUT,0.49,49000.0


In [48]:
# This is where round trips are added up for each security traded. But based on the fact that there are some trades that are not
# closed at the end of the trading competition. Round trips needs to be seperated into seperate rows. So that we dont miss out
# trades. That will be done in the next section . UPDATE : Avoiding it now for simplicity purpose
for indx,row in table.iterrows():
    if trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity.item() == 0:
        trade_analyzis.loc[trade_analyzis.security==row.Security,'number_of_trades']+= 1
        if row.Position=='BOT':
            trade_analyzis.loc[trade_analyzis.security==row.Security,'direction']='Long'
        else:
            trade_analyzis.loc[trade_analyzis.security==row.Security,'direction']='Short'
            
#             trade_analyzis[trade_analyzis.security==row.Security].number_of_trades+1
            
    trade_analyzis.loc[trade_analyzis.security==row.Security,'total_profit'] = row.Cash_flow
#     print((trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity)+ int(row.Quantity))
    trade_analyzis.loc[trade_analyzis.security==row.Security,'outstanding_quantity'] += int(row.Quantity)
#     trade_analyzis.loc[trade_analyzis.security==row.Security,'outstanding_quantity']+int(row.Quantity)
    print(trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity)
    
    if (trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity.item() != 0):
        trade_analyzis.loc[trade_analyzis.security==row.Security,'start_time'] = row.Time
    else:
        print("poda",trade_analyzis[trade_analyzis.security==row.Security].start_time.item())
        print("maire",row.Time )
        trade_analyzis.loc[trade_analyzis.security==row.Security,'average_holding_period']+=\
        ((row.Time- trade_analyzis.loc[trade_analyzis.security==row.Security,'start_time'])/datetime.timedelta(hours=1))
        trade_analyzis[trade_analyzis.security==row.Security].average_holding_period = 0


    

0    200
Name: outstanding_quantity, dtype: int64
0    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 10:10:19
maire 2016-10-11 13:50:28
1    300
Name: outstanding_quantity, dtype: int64
2    100
Name: outstanding_quantity, dtype: int64


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


1    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 14:11:19
maire 2016-10-11 14:27:21
3    10
Name: outstanding_quantity, dtype: int64
3    400
Name: outstanding_quantity, dtype: int64
2    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 14:22:10
maire 2016-10-11 14:43:11
3    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 14:31:19
maire 2016-10-11 14:54:37
4    1000
Name: outstanding_quantity, dtype: int64
5    1100
Name: outstanding_quantity, dtype: int64
5    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 12:48:58
maire 2016-10-12 13:00:17
6    1000
Name: outstanding_quantity, dtype: int64
6    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 13:05:11
maire 2016-10-12 13:10:01
7    630
Name: outstanding_quantity, dtype: int64
7   -630
Name: outstanding_quantity, dtype: int64
4    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 12:46:13
maire 2016-10-12 14:48:50
7    0
Name: outstanding_quantity, dtype: int64
poda 2

In [ ]:
trade_analyzis
trade_analyzis.total_profit.sum()
trade_analyzis.sum(axis=0)
trade_analyzis

In [ ]:
# trade_analyzis.loc[trade_analyzis.loc["VXX 100 21 OCT 16 34.5 PUT"].,average_holding_period]
trade_analyzis.loc[trade_analyzis.security=="VXX 100 14 OCT 16 34.5 PUT",'security']
# df.loc[df['column_name'] == some_value]
# trade_analyzis.[trade_analyzis==row.Security].average_holding_period = 0


In [32]:
row=table.iloc[1,:]
(row.Time- trade_analyzis[trade_analyzis.security==row.Security].start_time)
# row.Security

54   03:40:09
Name: start_time, dtype: timedelta64[ns]

In [33]:
# (row.Time- trade_analyzis.loc[trade_analyzis.security==row.Security,'start_time'])/datetime.timedelta(hours=1)

In [15]:
import ystockquote
print(ystockquote.get_historical_prices('CHK', '2013-01-03', '2013-01-08'))

{'2013-01-08': {'Open': '17.500001', 'High': '17.550001', 'Low': '16.850001', 'Close': '16.880001', 'Volume': '15977900', 'Adj Close': '15.308872'}, '2013-01-07': {'Open': '17.39', 'High': '17.690001', 'Low': '17.290001', 'Close': '17.620001', 'Volume': '13597900', 'Adj Close': '15.979997'}, '2013-01-04': {'Open': '16.870001', 'High': '17.46', 'Low': '16.800001', 'Close': '17.45', 'Volume': '13450300', 'Adj Close': '15.825819'}, '2013-01-03': {'Open': '16.600001', 'High': '17.030001', 'Low': '16.410001', 'Close': '16.770001', 'Volume': '13666200', 'Adj Close': '15.209111'}}


In [51]:
# import re

# mystr = 'This is a string, with words!'
# wordList = re.sub("[^\w]", " ",  mystr).split()

# trade_analyzis.underlying = re.sub("[^\w]", " ",  trade_analyzis.security).split()[0]
trade_analyzis['underlying']=trade_analyzis.security.str.extract('([A-Z]\w{0,})', expand=True)

In [52]:
trade_analyzis

,average_holding_period,average_profit,direction,number_of_trades,outstanding_quantity,security,start_time,total_profit,underlying
0,3.669167,0,Long,1,0,VXX 100 28 OCT 16 34.5 CALL,2016-10-11 10:10:19,-37200.0,VXX
1,24.513056,0,Long,2,0,VXX 100 21 OCT 16 34.5 PUT,2016-10-17 14:41:49,-81600.0,VXX
2,0.350278,0,Long,1,0,SPY 100 14 OCT 16 213.0 CALL,2016-10-11 14:22:10,-14200.0,SPY
3,0.388333,0,Long,1,0,VXX 100 14 OCT 16 34.5 PUT,2016-10-11 14:31:19,-40800.0,VXX
4,2.043611,0,Long,1,0,VXX 100 14 OCT 16 33.5 PUT,2016-10-12 12:46:13,-36000.0,VXX
5,0.188611,0,Long,1,0,UVXY 100 14 OCT 16 16.0 PUT,2016-10-12 12:48:58,-49500.0,UVXY
6,0.080556,0,Long,1,0,UVXY 100 14 OCT 16 17.0 CALL,2016-10-12 13:05:11,-45000.0,UVXY
7,0.157222,0,Long,1,0,GDX 100 21 OCT 16 23.0 CALL,2016-10-12 14:47:57,44100.0,GDX
8,20.815833,0,Long,1,0,NEM 100 21 OCT 16 34.5 PUT,2016-10-12 14:57:50,-44500.0,NEM
9,1.537222,0,Long,1,0,VXX 100 14 OCT 16 35.5 PUT,2016-10-13 11:36:43,-72000.0,VXX


In [41]:
# trade_analyzis.shape

(55, 9)

In [46]:
# trade_analyzis[trade_analyzis.outstanding_quantity!=0]
trade_analyzis[trade_analyzis.number_of_trades>1]


,average_holding_period,average_profit,end_time,number_of_trades,outstanding_quantity,security,start_time,total_profit,underlying
3,0.123056,0,0,2,200,TSLA 100 11 NOV 16 190.0 PUT,2016-11-04 13:27:59,92000.0,TSLA
31,0.648611,0,0,2,0,CHK 100 21 OCT 16 6.5 PUT,2016-10-21 13:19:33,-40000.0,CHK
36,24.513056,0,0,2,0,VXX 100 21 OCT 16 34.5 PUT,2016-10-17 14:41:49,-81600.0,VXX


In [64]:
# removing columns with outstanding quantity
trade=trade_analyzis[trade_analyzis.outstanding_quantity==0]
trade.sort_values(by='start_time',inplace=True)
trade.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,average_holding_period,average_profit,direction,number_of_trades,outstanding_quantity,security,start_time,total_profit,underlying
0,3.669167,0,Long,1,0,VXX 100 28 OCT 16 34.5 CALL,2016-10-11 10:10:19,-37200.0,VXX
2,0.350278,0,Long,1,0,SPY 100 14 OCT 16 213.0 CALL,2016-10-11 14:22:10,-14200.0,SPY
3,0.388333,0,Long,1,0,VXX 100 14 OCT 16 34.5 PUT,2016-10-11 14:31:19,-40800.0,VXX
4,2.043611,0,Long,1,0,VXX 100 14 OCT 16 33.5 PUT,2016-10-12 12:46:13,-36000.0,VXX
5,0.188611,0,Long,1,0,UVXY 100 14 OCT 16 16.0 PUT,2016-10-12 12:48:58,-49500.0,UVXY


In [65]:
# Changing the sign of profit due to the cashflow notation used
trade.loc[:,'total_profit']=trade.total_profit*-1
trade.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,average_holding_period,average_profit,direction,number_of_trades,outstanding_quantity,security,start_time,total_profit,underlying
0,3.669167,0,Long,1,0,VXX 100 28 OCT 16 34.5 CALL,2016-10-11 10:10:19,37200.0,VXX
2,0.350278,0,Long,1,0,SPY 100 14 OCT 16 213.0 CALL,2016-10-11 14:22:10,14200.0,SPY
3,0.388333,0,Long,1,0,VXX 100 14 OCT 16 34.5 PUT,2016-10-11 14:31:19,40800.0,VXX
4,2.043611,0,Long,1,0,VXX 100 14 OCT 16 33.5 PUT,2016-10-12 12:46:13,36000.0,VXX
5,0.188611,0,Long,1,0,UVXY 100 14 OCT 16 16.0 PUT,2016-10-12 12:48:58,49500.0,UVXY


In [90]:
melted=pd.DataFrame(trade.groupby('underlying').sum()).reset_index()
melted
# 

,underlying,average_holding_period,average_profit,number_of_trades,outstanding_quantity,total_profit
0,AAPL,22.613333,0,1,0,212800.0
1,AET,3.255556,0,1,0,9600.0
2,APC,21.965556,0,1,0,33300.0
3,CHK,48.777778,0,3,0,104500.0
4,GDX,262.440000,0,8,0,275200.0
5,GLD,72.786667,0,1,0,29600.0
6,HP,21.609167,0,1,0,46550.0
7,KO,216.087222,0,1,0,1890.0
8,MYL,2.524167,0,1,0,17400.0
9,NEM,187.568333,0,5,0,286900.0


In [80]:
melted.loc[:,'average_profit']=round(melted.total_profit/melted.number_of_trades,2)
melted

,average_holding_period,average_profit,number_of_trades,outstanding_quantity,total_profit
underlying,,,,,
AAPL,22.613333,212800.00,1,0,212800.0
AET,3.255556,9600.00,1,0,9600.0
APC,21.965556,33300.00,1,0,33300.0
CHK,48.777778,34833.33,3,0,104500.0
GDX,262.440000,34400.00,8,0,275200.0
GLD,72.786667,29600.00,1,0,29600.0
HP,21.609167,46550.00,1,0,46550.0
KO,216.087222,1890.00,1,0,1890.0
MYL,2.524167,17400.00,1,0,17400.0


In [96]:
# import matplotlib.pyplot as plt
from bokeh.charts import Bar, output_file, BoxPlot
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show 
from bokeh.sampledata.autompg import autompg as df
output_notebook()
p = Bar(melted, 'underlying', values='average_holding_period', title="Total MPG by CYL")

# output_file("bar.html")
show(p,notebook_handle=True)


Loading BokehJS ...

In [105]:
p = BoxPlot(trade, values='total_profit', label='underlying',
            title="MPG Summary (grouped by CYL)",legend=False)
show(p)
# from bokeh.sampledata.autompg import autompg as df
# df

In [108]:
# from bokeh.charts import Donut, show
# data = pd.Series(pd.Series(melted.total_profit), index = pd.Series(melted.underlying))
# pie_chart = Donut(data)
# show(pie_chart)
from bokeh.charts import Donut, show
data = pd.Series(pd.Series(melted.total_profit), index = pd.Series(melted.underlying))
pie_chart = Donut(data)
show(pie_chart)